<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ปฏิบัติการครั้งที่ 9

ดาวน์โหลดข้อมูลอุณหภูมิสูงสุดรายวันปี 1998-2019 จาก link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/CM_temp.csv

1. เปลี่ยนจากข้อมูลรายวันให้เป็นข้อมูลค่าเฉลี่ยรายเดือน
2. จงแบ่งข้อมูลโดยให้ Test set เป็นข้อมูลของ 2 ปีสุดท้ายและ Training set เป็นข้อมูลที่เหลือ
3. ลองสร้างโมเดล SARIMA อันดับต่างๆ แล้วระบุโมเดลที่ดีที่สุด
4. จงแสดงแผนภาพของข้อมูลจริงกับข้อมูลที่ได้ในข้อ 3

In [ ]:
# ONLY RUN THIS WHEN THERE IS AN ERROR
!pip install --user statsmodels==0.13.1

In [ ]:
!wget -O CM_temp.csv https://donlapark.pages.dev/229351/data/CM_temp.csv

In [ ]:
!pip3 install pmdarima

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
data = pd.read_csv("CM_temp.csv", parse_dates=["Date"], index_col="Date")

data.head()

ข้อมูล (สมมติว่าชื่อ `data`) ที่ load มาเป็นข้อมูลรายวัน ทำการหาค่าเฉลี่ยรายเดือนด้วยคำสั่ง

In [ ]:
y = data['MaxTemp'].resample("M").mean()

In [ ]:
def rmse(y_true,y_pred):
  # Inputs
  # y_true: actual value, y_pred: predicted values
  # Output: RMSE between y_true and y_pred
  return np.sqrt(np.mean(np.square(y_true-y_pred)))

In [ ]:
train = y[:-24]
test = y[-24:]

print(test)

In [ ]:
adf_result = adfuller(train.dropna())
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])

kpss_result = kpss(train.dropna(), regression="c")
print("KPSS Statistic:", kpss_result[0])
print("p-value:", kpss_result[1])

In [ ]:
model_auto = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
print(model_auto.summary())

In [ ]:
p, d, q = model_auto.order
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()

print(model_fit.summary())

In [ ]:
forecast = model_fit.forecast(steps=len(test))

error = rmse(test, forecast)
print("Test RMSE:", error)

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(train, label="Train")
plt.plot(test, label="Test", color="orange")
plt.plot(test.index, forecast, label="Forecast", color="green")
plt.legend()
plt.title("ARIMA Forecast vs Actual")
plt.show()